In [1]:
from pyspark import  SparkContext,SparkConf
sc = SparkContext( )                                                                                                                                                                         

In [2]:
#读文件，存储在hdfs的/data目录下
user_data = sc.textFile("/data/ml-100k/u.data")
#取第一行,用户id，影片id， 评分，时间戳
print user_data.first()
rawRatings = user_data.map(lambda x: x.split('\t'))
rawRatings.take(5)

SyntaxError: invalid syntax (<ipython-input-2-baca9375c02e>, line 4)

In [95]:
from pyspark.mllib.recommendation import Rating, ALS
#
ratings = rawRatings.map(lambda x: Rating(int(x[0]),int(x[1]),float(x[2])))
print ratings.take(5)

[Rating(user=196, product=242, rating=3.0), Rating(user=186, product=302, rating=3.0), Rating(user=22, product=377, rating=1.0), Rating(user=244, product=51, rating=2.0), Rating(user=166, product=346, rating=1.0)]


In [86]:
#rank、iterations和lambda参数的值分别为50、10和0.01
model = ALS.train(ratings, 50, 10, 0.01)
userFeatures = model.userFeatures()
produceFeatures = model.productFeatures()
print userFeatures.count(), produceFeatures.count()


943 1682


In [87]:
#预测用户对电影的评分
predictRating = model.predict(789,123)
print predictRating
#获取用户评分最高的n个物品
topKRecs = model.recommendProducts(789,10)
for rec in topKRecs:
    print rec

3.72708593715
Rating(user=789, product=56, rating=5.928617614704717)
Rating(user=789, product=488, rating=5.321371873378474)
Rating(user=789, product=156, rating=5.302823174330343)
Rating(user=789, product=86, rating=5.228895000126937)
Rating(user=789, product=475, rating=5.088201342651726)
Rating(user=789, product=195, rating=5.08163768546423)
Rating(user=789, product=47, rating=5.059224268426972)
Rating(user=789, product=129, rating=5.030381578142392)
Rating(user=789, product=127, rating=5.008146973830464)
Rating(user=789, product=100, rating=4.970468607026141)


In [88]:
#查看某一用户评级过的电影
moviesForUser = ratings.groupBy(lambda x : x.user).mapValues(list).lookup(789)
for i in sorted(moviesForUser[0],key=lambda x : x.rating,reverse=True):
    print i.product, i.rating

127 5.0
475 5.0
9 5.0
50 5.0
150 5.0
276 5.0
129 5.0
100 5.0
741 5.0
1012 4.0
93 4.0
293 4.0
181 4.0
1008 4.0
508 4.0
1007 4.0
124 4.0
1161 3.0
294 3.0
1 3.0
284 3.0
1017 3.0
111 3.0
742 3.0
248 3.0
249 3.0
591 3.0
288 3.0
762 3.0
628 3.0
137 2.0
151 2.0
286 1.0


In [89]:
#load movie info
movies = sc.textFile("/data/ml-100k/u.item")
titles = movies.map(lambda line: (int(line.split('|')[0]),line.split('|')[1])).collectAsMap()
#print titles.values()
#print movies.take(5)
for i,rec in enumerate(topKRecs):
    print 'rank:'+str(i)+' '+str(titles[rec.product])+':'+str(rec.rating)

rank:0 Pulp Fiction (1994):5.9286176147
rank:1 Sunset Blvd. (1950):5.32137187338
rank:2 Reservoir Dogs (1992):5.30282317433
rank:3 Remains of the Day, The (1993):5.22889500013
rank:4 Trainspotting (1996):5.08820134265
rank:5 Terminator, The (1984):5.08163768546
rank:6 Ed Wood (1994):5.05922426843
rank:7 Bound (1996):5.03038157814
rank:8 Godfather, The (1972):5.00814697383
rank:9 Fargo (1996):4.97046860703


In [90]:
import numpy as np
#cosin similary
def cosSimilarity(x,y):
    #norm ||A||_F = [\sum_{i,j} abs(a_{i,j})^2]^{1/2}
     return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
itemId = 123
itemFactor = model.productFeatures().lookup(itemId)[0]
#print itemFactor
sims = model.productFeatures().map(lambda (movieID, factor):(movieID, cosSimilarity(np.array(factor), np.array(itemFactor))))
print sims.take(2)

#desent sort
#sims.sortBy(lambda (x,y):y,ascending=False).take(10)
simsSorted = sims.sortBy(lambda (x,y):y,ascending=False)
print simsSorted.take(5)



[(4, 0.70832146828666498), (8, 0.70586454886399541)]
[(123, 1.0), (232, 0.83396609111740916), (471, 0.83127531828667356), (234, 0.81320819368222153), (28, 0.80862052795066497)]


In [91]:
print moviesForUser[0][0]
actual = moviesForUser[0][0];
realRating = actual.rating;
preRating = model.predict(789, actual.product)
squaredError = np.power(realRating-preRating,2)
print "real=",realRating,"preRating=", preRating,"err=",squaredError

Rating(user=789, product=1012, rating=4.0)
real= 4.0 preRating= 4.01251867655 err= 0.000156717262495


In [97]:
userProducts = ratings.map(lambda rating:(rating.user, rating.product))
print 'real score:',userProducts.take(5)

#预测所有用户对电影的相应评分
print model.predictAll(userProducts).collect()[0]
predictions = model.predictAll(userProducts).map(lambda rating:((rating.user,rating.product), rating.rating))
print 'predict score :',predictions.take(5)

ratingsAndPredictions = ratings.map(lambda rating:((rating.user,rating.product),rating.rating)).join(predictions)
print '组合预测的评分和实际的评分:',ratingsAndPredictions.take(5)

MSE = ratingsAndPredictions.map(lambda ((x,y),(m,n)):np.power(m-n,2)).reduce(lambda x,y:x+y)/ratingsAndPredictions.count() 
print '模型的均方误差:',MSE 
print '模型的均方根误差:',np.sqrt(MSE)

real score: [(196, 242), (186, 302), (22, 377), (244, 51), (166, 346)]
Rating(user=316, product=1084, rating=3.976026493965973)
predict score : [((316, 1084), 3.976026493965973), ((504, 1084), 3.9701833573179384), ((424, 1084), 4.94011247511074), ((541, 1084), 4.046125074732896), ((181, 1084), 2.054232861025692)]
组合预测的评分和实际的评分: [((711, 707), (5.0, 5.051851837818666)), ((650, 622), (3.0, 2.96665352309867)), ((472, 584), (1.0, 1.5990634734057037)), ((752, 316), (3.0, 2.937980478079039)), ((18, 428), (3.0, 3.3934276676079946))]
模型的均方误差: 0.0844192690433
模型的均方根误差: 0.290549942425


In [96]:
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.mllib.evaluation import RankingMetrics
predictedAndTrue = ratingsAndPredictions.map(lambda ((userId,product),(predicted, actual))
      :(predicted,actual))
print predictedAndTrue.take(5)
regressionMetrics = RegressionMetrics(predictedAndTrue)
print "均方误差 = %f"%regressionMetrics.meanSquaredError
print "均方根误差 = %f"% regressionMetrics.rootMeanSquaredError

[(5.0, 5.051851837818666), (3.0, 2.96665352309867), (1.0, 1.5990634734057037), (3.0, 2.937980478079039), (3.0, 3.3934276676079946)]
均方误差 = 0.084419
均方根误差 = 0.290550
